In [1]:
from tensorflow import keras
model = keras.models.load_model('drive/MyDrive/00AI/RNN_model')

In [2]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 10)            117530    
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 11753)             1187053   
                                                                 
Total params: 1,348,983
Trainable params: 1,348,983
Non-trainable params: 0
_________________________________________________________________


In [3]:
import os
import pandas as pd
all_headlines=[]
for filename in os.listdir('drive/MyDrive/00AI/articles/'):
  if 'Articles' in filename:
    headlines_df=pd.read_csv('drive/MyDrive/00AI/articles/'+filename)
    all_headlines.extend(list(headlines_df.headline.values))
all_headlines=[h for h in all_headlines if h !="Unknown"]

In [4]:
import tensorflow as tf
tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters= '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    analyzer=None)

In [5]:
tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words=len(tokenizer.word_index)+1
print('Total words',total_words)

Total words 11753


In [9]:
input_sequences=[]
for line in all_headlines:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    partial_sequence = token_list[:i+1]
    input_sequences.append(partial_sequence)

In [11]:
from keras.utils.data_utils import pad_sequences
import numpy as np
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [6]:
def predict_next_token(seed_text):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
  prediction=np.argmax(model.predict(token_list,verbose=0),axis=-1)
  return prediction

In [7]:
def generate_headline(seed_text,next_words=1):
  for _ in range(next_words):
    prediction=predict_next_token(seed_text)
    next_word=tokenizer.sequences_to_texts([prediction])[0]
    seed_text+=" " +next_word
  return seed_text.title()

In [12]:
seed_str=[
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']
for seed in seed_str:
  print(generate_headline(seed,next_words=10))

Washington Dc Is Already Signal A New York Times 200 000 Antelopes Since
Today In New York With Driving At A Cloud Case Of His Lips ’
The School District Has Open Until Its Government Fought Filling In The United States
Crime Has Become A New Lot In Deportations Words In The Air House
